In [5]:
import pickle
import networkx as nx
import pandas as pd
import os
import numpy as np
import statsmodels.api as sm
from statsmodels.regression.linear_model import OLSResults
import pickle
from tqdm import tqdm
from pprint import pprint
import yfinance as yf
from tqdm import tqdm, trange

In [6]:
with open('adj_mat.pkl', 'rb') as f:
    G = pickle.load(f)

In [10]:
# tickers = list(G.nodes)
tickers = pd.read_csv('tickers.csv',header=None)[0].to_list()
# tickers

### Data Aquisition

### Preprocessing
- Due to data limitation, we could only get daily return data. Thus, the realized covariance is calculated using daily return data, while the t-1 is equal to the previous month data

### HAR-DRD Model

In [11]:
def check_symmetric(a, rtol=1e-05, atol=1e-08):
    return np.allclose(a, a.T, rtol=rtol, atol=atol)

def is_pos_def(x):
    return np.all(np.linalg.eigvals(x) > 0)
    
class StockEngine():
    def __init__(self, tickers, start_date="2007-01-01", end_date="2022-12-31"):
        # Raw Data From yFinance
        self.raw_data = yf.download(' '.join(tickers), 
                                    start=start_date,
                                    end=end_date,
                                    group_by='ticker', 
                                    interval = "1d",
                                    actions=False)
        # Return Data
        ret_df = pd.DataFrame()
        for ticker in tqdm(tickers):
            ret_df[ticker] = (self.raw_data[ticker]['Adj Close']/self.raw_data[ticker]['Adj Close'].shift(1)-1)
        ## temporary solution
        self.ret_df = ret_df.drop(['DVA', 'ULTA'],axis=1).dropna()
        ret_df = ret_df[sorted(list(ret_df))]
        self.tickers = list(self.ret_df)

        # Year Month
        year_month_list = []
        for year in np.arange(int(start_date[:4]),int(end_date[:4])+1):
            for month in np.arange(1,13):
              year_month_list.append(str(year)+'-'+str(month).zfill(2))
        self.year_month_list = year_month_list
        self.trading_day_list = list(self.ret_df.index.strftime('%Y-%m-%d'))

        # load connection matrix
        with open('Wmat.npy', 'rb') as f:
            W = np.load(f)
            Wfull = np.load(f)
        self.W = W
        self.Wfull = Wfull
    
    def get_shift_month(self, year_month, diff):
        """
        Shift year month
        """
        idx = self.year_month_list.index(year_month) + diff
        assert(idx>=0)
        return self.year_month_list[idx]

    def get_shift_tday(self, tday, diff):
        """
        Shift trading days
        """
        idx = self.trading_day_list.index(tday) + diff
        assert(idx>=0)
        return self.trading_day_list[idx]

    def get_HAR_DRD(self, tday):
        """
        Get HAR DRD for the 21 days ending on given trading day
        """
        r = self.ret_df.loc[self.get_shift_tday(tday,-21*1+1):self.get_shift_tday(tday,0)]
        # print('Last Trading Day', self.get_shift_tday(tday,-21*1+1))
        # Ht = np.cov(r.T)
        # Rt = np.corrcoef(r.T)
        # RVt = np.diag(Ht)
        # Dt = np.diag(np.sqrt(RVt))

        Ht = np.zeros((r.shape[1], r.shape[1]))
        for i in range(r.shape[0]):
            Ht+=np.outer(r.iloc[i].values, r.iloc[i].values)
        RVt = np.diag(Ht)
        Dt = np.diag(np.sqrt(RVt))
        Dt_inverse = np.diag(1/np.sqrt(RVt))
        Rt = Dt_inverse @ Ht @ Dt_inverse

        return Ht, Rt, Dt, RVt

    def get_HAR_DRD_data(self, tday):
        """
        Get HAR DRD training data for the trading month (21 days) ending on a given trading day
        """
        Ht, Rt, Dt, RVt = self.get_HAR_DRD(tday)
        Ht1, Rt1, Dt1, RVt1 = self.get_HAR_DRD(self.get_shift_tday(tday, -21))

        il = np.tril_indices(Rt.shape[0], -1)
        xt = Rt[il]
        xt1 = Rt1[il]

        RVt25 = np.zeros_like(RVt)
        xt25 = np.zeros_like(xt)

        for i in range(2, 6):
            prev_tday = self.get_shift_tday(tday, -i*21)
            # print('Previous Trading Day:',prev_tday)
            Htp, Rtp, Dtp, RVtp = self.get_HAR_DRD(prev_tday)
            # print(prev_year_month)
            il = np.tril_indices(Rtp.shape[0], -1)
            xtp = Rtp[il]

            # Add up to explanatory variables
            RVt25 += 0.25*RVtp
            xt25 += 0.25*xtp

        RVt626 = np.zeros_like(RVt)
        xt626 = np.zeros_like(xt)
        # print('#####')
        for i in range(6, 23):
            prev_tday = self.get_shift_tday(tday, -i*21)
            # print('Previous Trading Day:',prev_tday)
            Htp, Rtp, Dtp, RVtp = self.get_HAR_DRD(prev_tday)
            il = np.tril_indices(Rtp.shape[0], -1)
            xtp = Rtp[il]

            # Add up to explanatory variables
            RVt626 += RVtp/17
            xt626 += xtp/17
        return RVt, RVt1, RVt25, RVt626, xt, xt1, xt25, xt626

    def get_GHAR_DRD_data(self, tday):
        RVt, RVt1, RVt25, RVt626, xt, xt1, xt25, xt626 = self.get_HAR_DRD_data(tday)
        WRVt, WRVt1, WRVt25, WRVt626 = self.W @ RVt, self.W @ RVt1, self.W @ RVt25, self.W @ RVt626

        return RVt, RVt1, RVt25, RVt626, xt, xt1, xt25, xt626, WRVt, WRVt1, WRVt25, WRVt626

    def get_GHAR_DRD_F_data(self, tday):
        RVt, RVt1, RVt25, RVt626, xt, xt1, xt25, xt626 = self.get_HAR_DRD_data(tday)
        WRVt, WRVt1, WRVt25, WRVt626 = self.Wfull @ RVt, self.Wfull @ RVt1, self.Wfull @ RVt25, self.Wfull @ RVt626

        return RVt, RVt1, RVt25, RVt626, xt, xt1, xt25, xt626, WRVt, WRVt1, WRVt25, WRVt626

In [12]:
engine = StockEngine(tickers, start_date="2009-01-01", end_date="2022-12-31")

[*********************100%***********************]  88 of 88 completed


100%|██████████| 88/88 [00:00<00:00, 214.95it/s]


### HAR-DRD coefficients estimations (Daily)

In [15]:
month_start_list = []
year_month_start_idx = engine.year_month_list.index('2012-01')
trading_day_list = np.array(engine.trading_day_list)
for idx in range(year_month_start_idx, len(engine.year_month_list)):
    month_start_list.append(trading_day_list[trading_day_list>=engine.year_month_list[idx]][0])
month_start_list = np.array(month_start_list)

In [16]:
def generate_model_train_HAR_DRD(tday, interval=1):
    # Obtain training data
    RVt_list, RVt1_list, RVt25_list, RVt626_list, xt_list, xt1_list, xt25_list, xt626_list = [],[],[],[],[],[],[],[]
    for i in trange(1, 251, interval):
        ptday = engine.get_shift_tday(tday, -i)
        RVt, RVt1, RVt25, RVt626, xt, xt1, xt25, xt626 = engine.get_HAR_DRD_data(ptday)
        RVt_list.append(RVt)
        RVt1_list.append(RVt1)
        RVt25_list.append(RVt25)
        RVt626_list.append(RVt626)
        xt_list.append(xt)
        xt1_list.append(xt1)
        xt25_list.append(xt25)
        xt626_list.append(xt626)

    RVt_data = pd.DataFrame({'RVt':np.concatenate(RVt_list, axis=0),
                           'RVt1':np.concatenate(RVt1_list, axis=0),
                           'RVt25':np.concatenate(RVt25_list, axis=0),
                           'RVt626':np.concatenate(RVt626_list, axis=0)})

    # linear regression to estimate coefficients
    cols = list(RVt_data)
    cols.remove('RVt')
    Y = RVt_data['RVt']
    X = RVt_data[cols]
    X = sm.add_constant(X)
    IS_RV_model = sm.OLS(Y,X).fit()

    xt_data = pd.DataFrame({'xt':np.concatenate(xt_list, axis=0),
                        'xt1':np.concatenate(xt1_list, axis=0),
                        'xt25':np.concatenate(xt25_list, axis=0),
                        'xt626':np.concatenate(xt626_list, axis=0)})

    cols = list(xt_data)
    cols.remove('xt')
    Y = xt_data['xt']
    X = xt_data[cols]
    X = sm.add_constant(X)
    IS_x_model = sm.OLS(Y,X).fit()
    return IS_RV_model, IS_x_model

In [20]:
for tday in month_start_list:
    IS_RV_model, IS_x_model = generate_model_train_HAR_DRD(tday, interval=1)
    IS_RV_model.save(f'model/IS_RV_{tday}.pkl')
    IS_x_model.save(f'model/IS_x_{tday}.pkl')
    # break

100%|██████████| 250/250 [01:11<00:00,  3.49it/s]
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


### GHAR-DRD


In [22]:
def generate_model_train_GHAR_DRD(tday, interval=1):
    # Obtain training data
    RVt_list, RVt1_list, RVt25_list, RVt626_list, xt_list, xt1_list, xt25_list, xt626_list = [],[],[],[],[],[],[],[]
    WRVt1_list, WRVt25_list, WRVt626_list = [],[],[] 
    for i in trange(1, 251, interval):
        ptday = engine.get_shift_tday(tday, -i)
        RVt, RVt1, RVt25, RVt626, xt, xt1, xt25, xt626, _, WRVt1, WRVt25, WRVt626 = engine.get_GHAR_DRD_data(ptday)
        RVt_list.append(RVt)
        RVt1_list.append(RVt1)
        RVt25_list.append(RVt25)
        RVt626_list.append(RVt626)
        xt_list.append(xt)
        xt1_list.append(xt1)
        xt25_list.append(xt25)
        xt626_list.append(xt626)
        WRVt1_list.append(WRVt1)
        WRVt25_list.append(WRVt25)
        WRVt626_list.append(WRVt626)

    RVt_data = pd.DataFrame({'RVt':np.concatenate(RVt_list, axis=0),
                           'RVt1':np.concatenate(RVt1_list, axis=0),
                           'RVt25':np.concatenate(RVt25_list, axis=0),
                           'RVt626':np.concatenate(RVt626_list, axis=0),
                           'WRVt1':np.concatenate(WRVt1_list, axis=0),
                           'WRVt25':np.concatenate(WRVt25_list, axis=0),
                           'WRVt626':np.concatenate(WRVt626_list, axis=0)})

    # linear regression to estimate coefficients
    cols = list(RVt_data)
    cols.remove('RVt')
    Y = RVt_data['RVt']
    X = RVt_data[cols]
    X = sm.add_constant(X)
    IS_WRV_model = sm.OLS(Y,X).fit()

    # xt_data = pd.DataFrame({'xt':np.concatenate(xt_list, axis=0),
    #                     'xt1':np.concatenate(xt1_list, axis=0),
    #                     'xt25':np.concatenate(xt25_list, axis=0),
    #                     'xt626':np.concatenate(xt626_list, axis=0)})

    # cols = list(xt_data)
    # cols.remove('xt')
    # Y = xt_data['xt']
    # X = xt_data[cols]
    # X = sm.add_constant(X)
    # IS_x_model = sm.OLS(Y,X).fit()
    return IS_WRV_model

In [23]:
IS_WRV_model = generate_model_train_GHAR_DRD(tday, interval=1)

100%|██████████| 250/250 [01:08<00:00,  3.63it/s]
/usr/local/lib/python3.8/dist-packages/statsmodels/tsa/tsatools.py:142: FutureWarning: In a future version of pandas all arguments of concat except for the argument 'objs' will be keyword-only
  x = pd.concat(x[::order], 1)


In [24]:
IS_WRV_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                    RVt   R-squared:                       0.343
Model:                            OLS   Adj. R-squared:                  0.343
Method:                 Least Squares   F-statistic:                     1872.
Date:                Mon, 12 Dec 2022   Prob (F-statistic):               0.00
Time:                        21:52:10   Log-Likelihood:                 69081.
No. Observations:               21500   AIC:                        -1.381e+05
Df Residuals:                   21493   BIC:                        -1.381e+05
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0071      0.000     31.276      0.000       0.007       0.008
RVt1           0.3812      0.011     35.627      0.000       0.360       0.402
RVt25          0.2853      0.020     14.125      0.000       0.246       0.325
RVt626        -0.3053      0.019    -16.012      0.000      -0.343      -0.268
WRVt1         -0.0536      0.014     -3.860      0.000      -0.081      -0.026
WRVt25         0.2917      0.028     10.267      0.000       0.236       0.347
WRVt626        0.4310      0.023     18.628      0.000       0.386       0.476
==============================================================================
Omnibus:                    18090.389   Durbin-Watson:                   1.228
Prob(Omnibus):                  0.000   Jarque-Bera (JB):           750676.128
Skew:                           3.844   Prob(JB):                         0.00
Kurtosis:                      30.908   Cond. No.                         533.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [25]:
for tday in month_start_list:
    IS_WRV_model = generate_model_train_GHAR_DRD(tday, interval=1)
    IS_WRV_model.save(f'model/IS_WRV_{tday}.pkl')
    # break